In [1]:
from keras.preprocessing import text
from keras.src.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [2]:
data = """Deep learning (also known as deep structured learning) is part of a
broader family of machine learning methods based on artificial neural networks
with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks,
deep reinforcement learning, recurrent neural networks, convolutional neural networks and
Transformers have been applied to fields including computer vision, speech recognition,
natural language processing, machine translation, bioinformatics, drug design,
medical image analysis, climate science, material inspection and board game programs,
where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()


Tokenization

In [4]:
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id=tokenizer.word_index

word2id['PAD']=0
id2word={v:k for k,v in word2id.items()}
wids=[[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size=len(word2id)
embed_size=100
window_size=2

print('Vocablury Size: ',vocab_size)
print('Vocablury Sample',list(word2id.items())[:10])

Vocablury Size:  75
Vocablury Sample [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


Generate Training Data


In [7]:
def generate_context_word_pairs(corpus,window_size,vocab_size):
  context_length=window_size*2

  for words in corpus:
    sentence_length=len(words)
    for index,word in enumerate(words):
      context_words=[]
      label_word=[]
      start=index-window_size
      end=index+window_size+1

      context_words.append([words[i]
                           for i in range(start,end)
                           if 0<= i < sentence_length
                            and i != index
                           ])
      label_word.append(words)

      x=pad_sequences(context_words,maxlen=context_length)
      y=np_utils.to_categorical(label_word,vocab_size)
      yield(x,y)

i=0

for x,y in generate_context_word_pairs(corpus=wids,window_size=window_size,vocab_size=vocab_size):
  if 0 not in x[0]:
      print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])

      if i== 10:
        break
      i+=1



In [8]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense,Embedding,Lambda

cbow=Sequential()
cbow.add(Embedding(input_dim=vocab_size,output_dim=embed_size,input_length=window_size*2))
cbow.add(Lambda(lambda x:K.mean(x,axis=1),output_shape=(embed_size,)))
cbow.add(Dense(vocab_size,activation='softmax'))
cbow.compile(loss='categorical_crossentropy',optimizer='rmsprop')

print(cbow.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [15]:
for epoch in range (1,6):
  loss = 0.
  i = 0

  for x, y in generate_context_word_pairs(corpus=wids,window_size=window_size,vocab_size=vocab_size):
    i+=1
    loss += cbow.train_on_batch(x, y)
    if i % 100000 == 0:
      print('Processed {} (context,word) pairs',format(i))

  print('Epoch:',epoch, '\tLoss:',loss)
  # print()




InvalidArgumentError: ignored

In [17]:
weights=cbow.get_weights()[0]
weights=weights[1:]
print(weights.shape)

pd.DataFrame(weights,index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.011489,0.034900,-0.014529,0.036893,-0.005741,0.038129,-0.017136,0.034847,0.035521,0.041286,...,-0.032081,-0.015888,0.046692,-0.002544,0.013998,0.048224,-0.041938,0.030689,0.004461,-0.012074
networks,0.043689,0.031130,-0.045475,-0.003715,0.014131,0.045994,0.007665,0.004509,0.013238,0.009411,...,0.023422,0.003615,-0.024998,-0.009586,-0.021255,-0.039742,-0.042658,-0.009833,0.025649,0.020671
neural,-0.012198,0.013414,-0.041940,0.010924,0.016286,0.017565,0.035700,0.049477,0.022679,0.020622,...,0.010563,0.034510,-0.037898,0.002381,-0.038906,0.037057,-0.041110,0.037474,-0.005856,-0.029484
and,0.034847,0.031989,-0.018679,-0.046261,-0.027598,0.025986,0.003411,-0.031687,-0.028556,0.046778,...,0.028736,0.038844,0.027414,0.023275,-0.026358,-0.018772,0.046000,0.044511,0.014165,0.034552
as,-0.035057,0.008613,0.004908,-0.026797,0.031014,0.021528,-0.004898,-0.005430,-0.014883,-0.048013,...,0.044822,-0.026619,-0.021703,-0.011468,0.009455,0.008233,-0.029379,-0.001347,-0.036210,0.010923


In [18]:
from sklearn.metrics.pairwise import euclidean_distances
distance_matrix=euclidean_distances(weights)
print(distance_matrix.shape)

similiar_words={search_term:[id2word[idx] for idx in distance_matrix[word2id[search_term]-1] .argsort()[1:6]+1]
                 for search_term in ['deep']}

similiar_words

(74, 74)


{'deep': ['surpassing', 'also', 'computer', 'reinforcement', 'have']}